# Generative Adversarial Networks (GANs) for MNIST Generation

## Overview

Generative Adversarial Networks represent a breakthrough in generative modeling through adversarial training. Two neural networks compete against each other: a Generator that creates fake data and a Discriminator that tries to detect fake data.

## Key Concepts

### 1. The Minimax Game
GANs solve a **minimax optimization problem**:

$$\min_G \max_D V(D, G) = \mathbb{E}_{x \sim p_{data}(x)}[\log D(x)] + \mathbb{E}_{z \sim p_z(z)}[\log(1 - D(G(z)))]$$

**Intuition**:
- **Discriminator (D)**: Wants to maximize the ability to distinguish real from fake
- **Generator (G)**: Wants to minimize the discriminator's ability to detect fakes

### 2. Nash Equilibrium
At equilibrium:
- **Generator**: Produces data indistinguishable from real data
- **Discriminator**: Cannot distinguish real from fake (outputs 0.5)
- **Optimal solution**: $p_g(x) = p_{data}(x)$ (generator distribution equals data distribution)

### 3. Training Dynamics
Unlike other generative models, GANs use **adversarial training**:

**Step 1**: Train Discriminator
- Maximize ability to classify real vs fake
- $\max_D [\log D(x_{real}) + \log(1 - D(G(z)))]$

**Step 2**: Train Generator  
- Minimize discriminator's ability to detect fakes
- $\min_G [\log(1 - D(G(z)))]$ or $\max_G [\log D(G(z))]$ (non-saturating)

### 4. Loss Function Analysis
**GAN losses are bounded** but behave differently:

- **Discriminator Loss**: Binary cross-entropy ∈ [0, ∞), typically [0, 2]
- **Generator Loss**: Binary cross-entropy ∈ [0, ∞), typically [0, 5]
- **Ideal Training**: Both losses should converge to $\log(2) ≈ 0.693$

### 5. No Explicit Likelihood
Unlike VAEs or flows, GANs **don't compute explicit likelihood**:
- Cannot evaluate $p(x)$ for given data
- Focus purely on sample quality
- Evaluation requires external metrics (FID, IS)

### 6. Direct Sampling
**Generation Process**: $z \sim p(z) \rightarrow G(z) \rightarrow x$
- Simple sampling from noise distribution (usually $\mathcal{N}(0, I)$)
- No complex inversion or reparameterization needed
- Can generate high-quality, sharp images

## Implementation: Imports and Data Preprocessing

Unlike flow models, GANs work perfectly with discrete pixel values and don't require dequantization:

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from keras.datasets.mnist import load_data
from torch.utils.data import DataLoader
from tqdm import tqdm
import matplotlib.pyplot as plt

torch.manual_seed(0)

# Load and normalize MNIST dataset - standard approach for GANs
print("Loading MNIST dataset...")
(trainX, trainY), (testX, testy) = load_data()

# Standard normalization for GANs: just scale to [0,1]
trainX = np.float32(trainX) / 255.0  # No dequantization needed
trainX = torch.tensor(trainX.reshape(-1, 28 * 28))

print(f"Dataset shape: {trainX.shape}")
print(f"Pixel value range: [{trainX.min():.3f}, {trainX.max():.3f}]")
print("Note: GANs work well with discrete pixel values - no dequantization needed!")

# Set device
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

## Generator Network: Creating Fake Data

The Generator's job is to transform random noise into realistic MNIST digits:

**Architecture Design**:
- **Input**: Random noise vector $z \sim \mathcal{N}(0, I)$ (dimension 100)
- **Output**: Fake image $G(z)$ (dimension 784 for flattened 28×28)
- **Activation**: Sigmoid to ensure output ∈ [0,1]

**Key Principles**:
- Deep enough to learn complex mappings
- ReLU activations for non-linearity
- Sigmoid final layer for valid pixel values

In [ ]:
class Generator(nn.Module):
    """
    Generator Network: z → x̂
    
    Transforms random noise into realistic MNIST digits
    
    Architecture: 
    noise(100) → hidden(256) → hidden(256) → hidden(256) → image(784)
    
    Goal: Fool the discriminator into thinking generated images are real
    """
    def __init__(self, noise_dim=100, hidden_dim=256, output_dim=784):
        super(Generator, self).__init__()
        
        self.noise_dim = noise_dim
        
        self.model = nn.Sequential(
            # Layer 1: Noise to hidden
            nn.Linear(noise_dim, hidden_dim),
            nn.ReLU(),
            
            # Layer 2-3: Hidden layers for complexity
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            
            # Output layer: Generate image
            nn.Linear(hidden_dim, output_dim),
            nn.Sigmoid()  # Ensure output ∈ [0,1] for valid pixels
        )
        
        print(f"Initialized Generator:")
        print(f"  Noise dimension: {noise_dim}")
        print(f"  Hidden dimension: {hidden_dim}")
        print(f"  Output dimension: {output_dim}")
    
    def forward(self, z):
        """
        Generate fake images from noise
        
        Args:
            z: Random noise [batch_size, noise_dim]
            
        Returns:
            fake_images: Generated images [batch_size, output_dim]
        """
        return self.model(z)

# Initialize generator
noise_dim = 100
generator = Generator(noise_dim=noise_dim).to(device)
print(f"\nGenerator parameters: {sum(p.numel() for p in generator.parameters()):,}")

# Test generator
print("\nTesting generator...")
with torch.no_grad():
    test_noise = torch.randn(5, noise_dim).to(device)
    fake_images = generator(test_noise)
    print(f"Input noise shape: {test_noise.shape}")
    print(f"Generated images shape: {fake_images.shape}")
    print(f"Generated pixel range: [{fake_images.min():.3f}, {fake_images.max():.3f}]")

## Discriminator Network: Real vs Fake Detection

The Discriminator's job is to distinguish between real MNIST digits and generator fakes:

**Architecture Design**:
- **Input**: Image (real or fake) with dimension 784
- **Output**: Probability that input is real ∈ [0,1]
- **Activation**: LeakyReLU (helps gradient flow) + Dropout (prevents overfitting)

**Key Principles**:
- Must be powerful enough to provide useful gradients to generator
- Must not be too powerful (would make generator training impossible)
- Dropout and LeakyReLU improve training stability

In [ ]:
class Discriminator(nn.Module):
    """
    Discriminator Network: x → [0,1]
    
    Distinguishes between real MNIST digits and generator fakes
    
    Architecture:
    image(784) → hidden(256) → hidden(256) → hidden(256) → probability(1)
    
    Goal: Maximize ability to detect fake images
    """
    def __init__(self, input_dim=784, hidden_dim=256):
        super(Discriminator, self).__init__()
        
        self.model = nn.Sequential(
            # Layer 1: Image to hidden
            nn.Linear(input_dim, hidden_dim),
            nn.LeakyReLU(0.2),  # LeakyReLU helps with gradient flow
            nn.Dropout(0.3),    # Dropout prevents discriminator overfitting
            
            # Layer 2-3: Hidden layers
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            nn.Linear(hidden_dim, hidden_dim),
            nn.LeakyReLU(0.2),
            nn.Dropout(0.3),
            
            # Output layer: Real/fake classification
            nn.Linear(hidden_dim, 1),
            nn.Sigmoid()  # Output ∈ [0,1]: 0=fake, 1=real
        )
        
        print(f"Initialized Discriminator:")
        print(f"  Input dimension: {input_dim}")
        print(f"  Hidden dimension: {hidden_dim}")
        print(f"  Uses LeakyReLU(0.2) and Dropout(0.3)")
    
    def forward(self, x):
        """
        Classify input as real or fake
        
        Args:
            x: Input images [batch_size, input_dim]
            
        Returns:
            probability: P(x is real) ∈ [0,1] [batch_size, 1]
        """
        return self.model(x)

# Initialize discriminator
discriminator = Discriminator().to(device)
print(f"\nDiscriminator parameters: {sum(p.numel() for p in discriminator.parameters()):,}")

# Test discriminator
print("\nTesting discriminator...")
with torch.no_grad():
    # Test with real data
    test_real = trainX[:5].to(device)
    real_predictions = discriminator(test_real)
    
    # Test with fake data
    test_noise = torch.randn(5, noise_dim).to(device)
    test_fake = generator(test_noise)
    fake_predictions = discriminator(test_fake)
    
    print(f"Real data predictions: {real_predictions.mean().item():.3f} (should be ~1.0 when trained)")
    print(f"Fake data predictions: {fake_predictions.mean().item():.3f} (should be ~0.0 when trained)")
    print(f"Initial difference: {real_predictions.mean().item() - fake_predictions.mean().item():.3f}")

## Adversarial Loss Functions: The Heart of GANs

GANs use adversarial loss functions that create the minimax game:

### Mathematical Foundation

**Discriminator Loss** (wants to maximize):
$$L_D = -\mathbb{E}_{x \sim p_{data}}[\log D(x)] - \mathbb{E}_{z \sim p_z}[\log(1 - D(G(z)))]$$

**Generator Loss** (non-saturating version):
$$L_G = -\mathbb{E}_{z \sim p_z}[\log D(G(z))]$$

### Implementation Details
- Use **Binary Cross Entropy** loss for both networks
- **Labels**: Real=1, Fake=0
- **Non-saturating**: Generator maximizes $\log D(G(z))$ instead of minimizing $\log(1-D(G(z)))$

In [ ]:
# Initialize loss function and optimizers
criterion = nn.BCELoss()  # Binary Cross Entropy for adversarial loss

# Optimizers: Adam with specific hyperparameters for GANs
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

print("=== GAN Loss Function Demonstration ===")

# Demonstrate loss computation
batch_size = 32
real_images = trainX[:batch_size].to(device)

# Create labels
real_labels = torch.ones(batch_size, 1).to(device)   # Real = 1
fake_labels = torch.zeros(batch_size, 1).to(device)  # Fake = 0

print(f"Label setup:")
print(f"  Real labels: {real_labels.mean().item():.1f} (all ones)")
print(f"  Fake labels: {fake_labels.mean().item():.1f} (all zeros)")

# Forward pass with current (untrained) networks
with torch.no_grad():
    # Discriminator on real data
    real_outputs = discriminator(real_images)
    d_loss_real = criterion(real_outputs, real_labels)
    
    # Generator creates fake data
    noise = torch.randn(batch_size, noise_dim).to(device)
    fake_images = generator(noise)
    
    # Discriminator on fake data
    fake_outputs = discriminator(fake_images)
    d_loss_fake = criterion(fake_outputs, fake_labels)
    
    # Generator loss (wants discriminator to output 1 for fakes)
    g_loss = criterion(fake_outputs, real_labels)  # Note: real_labels!
    
    # Total discriminator loss
    d_loss_total = d_loss_real + d_loss_fake

print(f"\nInitial loss values (before training):")
print(f"  D(real) = {real_outputs.mean().item():.3f} (should approach 1.0)")
print(f"  D(fake) = {fake_outputs.mean().item():.3f} (should approach 0.0)")
print(f"  Discriminator loss: {d_loss_total.item():.3f}")
print(f"  Generator loss: {g_loss.item():.3f}")

print(f"\nLoss interpretation:")
print(f"  When D(real) → 1 and D(fake) → 0: Discriminator is winning")
print(f"  When D(fake) → 1: Generator is winning")
print(f"  At equilibrium: D(real) ≈ D(fake) ≈ 0.5")
print(f"  Perfect GAN: Both losses converge to log(2) ≈ 0.693")

# Show typical GAN loss ranges
print(f"\nTypical GAN loss ranges during training:")
print(f"  Discriminator loss: 0.0 - 2.0 (lower = discriminator winning)")
print(f"  Generator loss: 0.0 - 5.0 (higher = discriminator winning)")
print(f"  Healthy training: Both losses fluctuate around 0.5 - 1.5")

## Training Dynamics: The Adversarial Dance

GAN training involves a delicate balance between two competing networks:

### Training Algorithm
**For each batch**:
1. **Train Discriminator**:
   - Forward pass on real data → maximize $\log D(x)$
   - Forward pass on fake data → maximize $\log(1-D(G(z)))$
   - Update discriminator parameters

2. **Train Generator**:
   - Generate fake data
   - Forward pass through discriminator → maximize $\log D(G(z))$
   - Update generator parameters

### Training Challenges
- **Mode Collapse**: Generator produces limited variety
- **Training Instability**: Loss oscillations, non-convergence
- **Gradient Vanishing**: When discriminator is too good
- **Evaluation**: No likelihood-based metrics

## Common GAN Training Challenges: What Can Go Wrong

GAN training is notoriously unstable and prone to various failure modes. Understanding these issues is crucial for successful GAN training:

### 1. Mode Collapse 🚨

**What it is**: Generator produces limited variety of outputs, "collapsing" to a few modes of the data distribution.

**Symptoms**:
- Generated images look very similar
- Generator loss decreases but diversity disappears
- Missing entire classes of data (e.g., only generates digits 1, 3, 7)

**Example Output Patterns**:
```
Normal: Generates all digits 0-9 with variety
Mode Collapse: Only generates digit 3 repeatedly, or cycles between 2-3 digits
```

**Why it happens**:
- Generator finds a few "easy" examples that fool the discriminator
- No incentive to explore the full data distribution
- Discriminator can't provide useful gradients for diversity

### 2. Discriminator Collapse

**What it is**: Discriminator becomes too powerful, perfectly distinguishing real from fake.

**Symptoms** (exactly what you're seeing):
```
D_loss= 0.000, G_loss=12.840, D(x)=1.000, D(G(z))=0.000
```
- Discriminator loss ≈ 0 (perfect classification)
- Generator loss very high (12+)
- D(real) = 1.0, D(fake) = 0.0 (perfect separation)

**Why it's bad**:
- Generator gets no useful gradients (discriminator says "definitely fake")
- Training stalls completely
- Generator cannot improve

**Common Causes**:
- Discriminator learning rate too high
- Discriminator architecture too powerful
- Generator architecture too weak
- Not enough generator updates per discriminator update

### 3. Training Instability

**Nash Equilibrium Problem**:
- GANs seek Nash equilibrium between two players
- Unlike single optimization, no guarantee of convergence
- Small changes can destabilize entire training

**Oscillating Losses**:
```
Epoch 10: G_loss=2.1, D_loss=0.3  # D winning
Epoch 11: G_loss=0.8, D_loss=1.9  # G winning  
Epoch 12: G_loss=3.2, D_loss=0.1  # D winning again
```

**Non-Convergent Behavior**:
- Losses never stabilize
- Generated samples don't improve
- Quality oscillates between good and terrible

### 4. Vanishing Gradients

**When Discriminator is Too Good**:
- $\log(1 - D(G(z))) \rightarrow -\infty$ when $D(G(z)) \rightarrow 1$
- Generator gradients vanish → no learning
- Training stagnates

**Solution - Non-Saturating Loss**:
- Instead of $\min_G \log(1 - D(G(z)))$
- Use $\max_G \log(D(G(z)))$ (what we implement)

### 5. Other Common Issues

**Hyperparameter Sensitivity**:
- Learning rates must be carefully balanced
- Architecture choices heavily impact stability
- Batch size affects training dynamics

**Evaluation Difficulty**:
- Loss values don't correlate with sample quality
- Need external metrics (FID, IS, visual inspection)
- Hard to know when to stop training

**Memory and Computational Cost**:
- Two networks instead of one
- Adversarial training more expensive than standard training

### Solutions and Mitigation Strategies

**For Discriminator Collapse** (your issue):
1. **Reduce discriminator learning rate**: `d_lr = 0.0001, g_lr = 0.0002`
2. **Train generator more frequently**: 2-3 G updates per D update
3. **Add noise to discriminator inputs**: `x + 0.1 * torch.randn_like(x)`
4. **Use label smoothing**: Real labels = 0.9, Fake labels = 0.1
5. **Reduce discriminator capacity**: Fewer layers or smaller hidden dims

**For Mode Collapse**:
1. **Minibatch discrimination**: Let discriminator see multiple samples
2. **Unrolled GANs**: Generator considers future discriminator updates
3. **Feature matching**: Generator matches statistics, not raw output

**For Training Instability**:
1. **WGAN**: Use Wasserstein distance instead of JS divergence
2. **Spectral normalization**: Constrain discriminator Lipschitz constant
3. **Progressive growing**: Start with low resolution, gradually increase
4. **Self-attention**: Improve long-range dependencies

**General Best Practices**:
1. **Monitor multiple metrics**: Not just loss values
2. **Save checkpoints frequently**: Training can suddenly diverge
3. **Use proven architectures**: DCGAN, StyleGAN patterns
4. **Patience**: GANs often need 100+ epochs to produce good results

### Example of Healthy GAN Training

**Good Loss Pattern**:
```
Epoch 1:  D_loss=1.2, G_loss=2.1, D(x)=0.8, D(G(z))=0.3
Epoch 10: D_loss=0.8, G_loss=1.5, D(x)=0.7, D(G(z))=0.4  
Epoch 50: D_loss=0.7, G_loss=0.9, D(x)=0.6, D(G(z))=0.5
```

**Indicators of Success**:
- Both losses converge to ≈ 0.693 (log 2)
- D(real) and D(fake) both approach 0.5
- Generated samples show good diversity
- Visual quality improves over time

Understanding these challenges helps you diagnose issues and adjust training accordingly!

In [ ]:
def generate_images_gan(generator, epoch, nb_data=10, noise_dim=100, device='cpu'):
    """
    Generate and display GAN sample images
    
    Process: z ~ N(0,I) → Generator → fake_images
    """
    generator.eval()
    with torch.no_grad():
        # Sample random noise
        noise = torch.randn(nb_data * nb_data, noise_dim).to(device)
        fake_images = generator(noise).cpu().numpy()
        
        fig, axs = plt.subplots(nb_data, nb_data, figsize=(10, 10))
        for i in range(nb_data):
            for j in range(nb_data):
                idx = i * nb_data + j
                axs[i, j].imshow(fake_images[idx].reshape(28, 28), cmap='gray')
                axs[i, j].set_xticks([])
                axs[i, j].set_yticks([])
        plt.suptitle(f'GAN Generated Images - Epoch {epoch}')
        plt.tight_layout()
        plt.show()
    generator.train()


def train_gan_improved(generator, discriminator, g_optimizer, d_optimizer, dataloader, 
                      nb_epochs=30, noise_dim=100, device='cpu'):
    """
    Improved GAN training with fixes for common issues:
    
    Fixes implemented:
    1. Label smoothing (prevents discriminator overconfidence)
    2. Noisy inputs (adds regularization to discriminator) 
    3. Different learning rates (balances G/D power)
    4. Multiple generator updates (helps with discriminator collapse)
    5. Feature matching loss (encourages diversity)
    """
    criterion = nn.BCELoss()
    mse_loss = nn.MSELoss()  # For feature matching
    
    g_losses = []
    d_losses = []
    
    # Training hyperparameters for stability
    label_smoothing = 0.1      # Real labels = 1-0.1, Fake labels = 0+0.1
    noise_factor = 0.05        # Add noise to discriminator inputs
    g_updates_per_d = 2        # Train generator more frequently
    
    print("=== Starting Improved GAN Training ===")
    print(f"Fixes enabled:")
    print(f"  - Label smoothing: {label_smoothing}")
    print(f"  - Input noise factor: {noise_factor}")
    print(f"  - Generator updates per D update: {g_updates_per_d}")
    print(f"  - Feature matching loss")
    
    for epoch in tqdm(range(nb_epochs), desc="Training Improved GAN"):
        epoch_g_loss = 0
        epoch_d_loss = 0
        
        for batch_idx, real_images in enumerate(dataloader):
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            
            # ===========================
            # Train Discriminator (once per batch)
            # ===========================
            d_optimizer.zero_grad()
            
            # FIX 1: Label smoothing - prevents overconfident discriminator
            real_labels = torch.ones(batch_size, 1).to(device) * (1.0 - label_smoothing)
            fake_labels = torch.zeros(batch_size, 1).to(device) + label_smoothing
            
            # FIX 2: Add noise to real images - regularizes discriminator
            real_images_noisy = real_images + noise_factor * torch.randn_like(real_images)
            real_images_noisy = torch.clamp(real_images_noisy, 0, 1)  # Keep in valid range
            
            # Real images: should output ~0.9 (not 1.0)
            real_outputs = discriminator(real_images_noisy)
            d_loss_real = criterion(real_outputs, real_labels)
            
            # Fake images: should output ~0.1 (not 0.0)
            noise = torch.randn(batch_size, noise_dim).to(device)
            fake_images = generator(noise)
            fake_outputs = discriminator(fake_images.detach())  # Detach to avoid generator gradients
            d_loss_fake = criterion(fake_outputs, fake_labels)
            
            # Total discriminator loss
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()
            
            # ===========================
            # Train Generator (multiple times per batch)
            # FIX 3: Multiple generator updates helps with discriminator collapse
            # ===========================
            for g_step in range(g_updates_per_d):
                g_optimizer.zero_grad()
                
                # Generate fake images and try to fool discriminator
                noise = torch.randn(batch_size, noise_dim).to(device)
                fake_images = generator(noise)
                fake_outputs = discriminator(fake_images)
                
                # Standard adversarial loss: want discriminator to output 1
                g_loss_adv = criterion(fake_outputs, torch.ones(batch_size, 1).to(device))
                
                # FIX 4: Feature matching loss - encourages diversity
                # Match statistics of intermediate features, not just final output
                with torch.no_grad():
                    real_features = discriminator.model[:-2](real_images)  # Features before final layer
                fake_features = discriminator.model[:-2](fake_images)
                g_loss_feature = mse_loss(fake_features.mean(0), real_features.mean(0))
                
                # Combined generator loss
                g_loss = g_loss_adv + 0.1 * g_loss_feature  # Weight feature matching
                
                g_loss.backward()
                g_optimizer.step()
            
            epoch_g_loss += g_loss.item()
            epoch_d_loss += d_loss.item()
            
            # Print loss components for monitoring - less frequent to avoid spam
            if batch_idx % 200 == 0:  # Every 200 batches instead of 100
                print(f"Epoch {epoch+1:3d}, Batch {batch_idx:3d}: "
                      f"D_loss={d_loss.item():6.3f}, G_loss={g_loss.item():6.3f}, "
                      f"D(x)={real_outputs.mean().item():.3f}, "
                      f"D(G(z))={fake_outputs.mean().item():.3f}")
                
                # FIX 5: Better monitoring - warn about potential issues
                d_real_mean = real_outputs.mean().item()
                d_fake_mean = fake_outputs.mean().item()
                
                if d_real_mean > 0.95 and d_fake_mean < 0.05:
                    print("  ⚠️  Discriminator dominating - consider reducing D learning rate")
                elif d_real_mean < 0.55 and d_fake_mean > 0.45:
                    print("  ⚠️  Generator dominating - discriminator may need help")
                elif 0.3 < d_real_mean < 0.7 and 0.3 < d_fake_mean < 0.7:
                    print("  ✓ Good balance between generator and discriminator")
        
        # Store average losses
        g_losses.append(epoch_g_loss / len(dataloader))
        d_losses.append(epoch_d_loss / len(dataloader))
        
        # Generate images every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"\nGenerating GAN images at epoch {epoch + 1}")
            generate_images_gan(generator, epoch + 1, device=device)
            
            # Additional diagnostics every 10 epochs
            with torch.no_grad():
                # Check diversity by generating many samples
                diversity_noise = torch.randn(100, noise_dim).to(device)
                diversity_samples = generator(diversity_noise)
                diversity_std = diversity_samples.std().item()
                
                print(f"Epoch {epoch + 1} Diagnostics:")
                print(f"  Generator diversity (std): {diversity_std:.4f}")
                print(f"  Average D(real): {d_real_mean:.3f}")
                print(f"  Average D(fake): {d_fake_mean:.3f}")
                
                if diversity_std < 0.1:
                    print("  ⚠️  Low diversity - possible mode collapse!")
                elif diversity_std > 0.3:
                    print("  ✓ Good sample diversity")
    
    return g_losses, d_losses


# Also create the original training function for comparison
def train_gan_vanilla(generator, discriminator, g_optimizer, d_optimizer, dataloader, 
                     nb_epochs=30, noise_dim=100, device='cpu'):
    """
    Original vanilla GAN training (prone to instability)
    Kept for educational comparison
    """
    criterion = nn.BCELoss()
    g_losses = []
    d_losses = []
    
    for epoch in tqdm(range(nb_epochs), desc="Training Vanilla GAN"):
        epoch_g_loss = 0
        epoch_d_loss = 0
        
        for batch_idx, real_images in enumerate(dataloader):
            batch_size = real_images.size(0)
            real_images = real_images.to(device)
            
            # Create labels (hard labels - problematic)
            real_labels = torch.ones(batch_size, 1).to(device)    # Hard 1.0
            fake_labels = torch.zeros(batch_size, 1).to(device)   # Hard 0.0
            
            # Train Discriminator
            d_optimizer.zero_grad()
            
            real_outputs = discriminator(real_images)  # No noise added
            d_loss_real = criterion(real_outputs, real_labels)
            
            noise = torch.randn(batch_size, noise_dim).to(device)
            fake_images = generator(noise)
            fake_outputs = discriminator(fake_images.detach())
            d_loss_fake = criterion(fake_outputs, fake_labels)
            
            d_loss = d_loss_real + d_loss_fake
            d_loss.backward()
            d_optimizer.step()
            
            # Train Generator (only once - problematic)
            g_optimizer.zero_grad()
            
            noise = torch.randn(batch_size, noise_dim).to(device)
            fake_images = generator(noise)
            fake_outputs = discriminator(fake_images)
            g_loss = criterion(fake_outputs, real_labels)  # Only adversarial loss
            
            g_loss.backward()
            g_optimizer.step()
            
            epoch_g_loss += g_loss.item()
            epoch_d_loss += d_loss.item()
        
        g_losses.append(epoch_g_loss / len(dataloader))
        d_losses.append(epoch_d_loss / len(dataloader))
        
        if (epoch + 1) % 10 == 0:
            generate_images_gan(generator, epoch + 1, device=device)
    
    return g_losses, d_losses

print("Training functions defined!")
print("Use train_gan_improved() for stable training with fixes")
print("Use train_gan_vanilla() for comparison (will likely fail)")

## Training the GAN Model

Let's train our GAN and observe the adversarial dynamics:

In [ ]:
# Training setup with improved hyperparameters
dataloader = DataLoader(trainX, batch_size=128, shuffle=True)

# FIX: Different learning rates for better balance  
# Discriminator gets slower learning rate to prevent dominance
g_optimizer = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer = optim.Adam(discriminator.parameters(), lr=0.0001, betas=(0.5, 0.999))  # Slower!

print("=== Improved GAN Training Setup ===")
print(f"Generator learning rate: 0.0002")
print(f"Discriminator learning rate: 0.0001 (slower to prevent dominance)")
print(f"Batch size: 128")
print(f"Training improvements enabled!")

# Test forward passes
print("\nTesting GAN forward passes...")
with torch.no_grad():
    test_noise = torch.randn(5, noise_dim).to(device)
    test_real = next(iter(dataloader))[:5].to(device)
    
    fake_test = generator(test_noise)
    real_pred = discriminator(test_real)
    fake_pred = discriminator(fake_test)
    
    print(f"Test results:")
    print(f"  Generator: {test_noise.shape} → {fake_test.shape}")
    print(f"  Discriminator on real: {real_pred.mean().item():.3f}")
    print(f"  Discriminator on fake: {fake_pred.mean().item():.3f}")

# Train improved GAN
print("\nStarting IMPROVED GAN training...")
print("This should be much more stable than vanilla GAN training!")

g_losses_improved, d_losses_improved = train_gan_improved(
    generator, discriminator, g_optimizer, d_optimizer, 
    dataloader, nb_epochs=50, device=device  # More epochs since it's more stable
)

print("\nImproved GAN training completed!")

# Optional: For educational comparison, you can also run vanilla training
# (Uncomment the lines below to see the difference)
"""
print("\n" + "="*50)
print("COMPARISON: Running Vanilla GAN (will likely fail)")
print("="*50)

# Reset networks for fair comparison
generator_vanilla = Generator(noise_dim=noise_dim).to(device)
discriminator_vanilla = Discriminator().to(device)
g_optimizer_vanilla = optim.Adam(generator_vanilla.parameters(), lr=0.0002, betas=(0.5, 0.999))
d_optimizer_vanilla = optim.Adam(discriminator_vanilla.parameters(), lr=0.0002, betas=(0.5, 0.999))

g_losses_vanilla, d_losses_vanilla = train_gan_vanilla(
    generator_vanilla, discriminator_vanilla, g_optimizer_vanilla, d_optimizer_vanilla,
    dataloader, nb_epochs=20, device=device
)
"""

## Analyzing GAN Training Dynamics

Let's visualize the training dynamics and understand what makes GANs challenging:

In [ ]:
# Plot training curves
print("=== GAN Training Analysis ===")

plt.figure(figsize=(15, 5))

# Plot 1: Loss curves
plt.subplot(1, 3, 1)
epochs = range(1, len(g_losses) + 1)
plt.plot(epochs, g_losses, label='Generator Loss', color='blue')
plt.plot(epochs, d_losses, label='Discriminator Loss', color='red')
plt.axhline(y=0.693, color='green', linestyle='--', alpha=0.7, label='log(2) ≈ 0.693')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('GAN Training Losses')
plt.legend()
plt.grid(True, alpha=0.3)

# Plot 2: Loss difference
plt.subplot(1, 3, 2)
loss_diff = np.array(g_losses) - np.array(d_losses)
plt.plot(epochs, loss_diff, color='purple')
plt.axhline(y=0, color='black', linestyle='-', alpha=0.7)
plt.xlabel('Epoch')
plt.ylabel('Generator Loss - Discriminator Loss')
plt.title('Training Balance')
plt.grid(True, alpha=0.3)

# Plot 3: Loss moving average
plt.subplot(1, 3, 3)
window = 3
if len(g_losses) >= window:
    g_smooth = np.convolve(g_losses, np.ones(window)/window, mode='valid')
    d_smooth = np.convolve(d_losses, np.ones(window)/window, mode='valid')
    smooth_epochs = range(window, len(g_losses) + 1)
    plt.plot(smooth_epochs, g_smooth, label='Generator (smoothed)', color='blue')
    plt.plot(smooth_epochs, d_smooth, label='Discriminator (smoothed)', color='red')
    plt.xlabel('Epoch')
    plt.ylabel('Smoothed Loss')
    plt.title('Smoothed Training Curves')
    plt.legend()
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Analysis
final_g_loss = g_losses[-1]
final_d_loss = d_losses[-1]
loss_ratio = final_g_loss / final_d_loss

print(f"\nTraining Analysis:")
print(f"  Final Generator Loss: {final_g_loss:.3f}")
print(f"  Final Discriminator Loss: {final_d_loss:.3f}")
print(f"  Loss Ratio (G/D): {loss_ratio:.3f}")
print(f"  Ideal target: Both ≈ 0.693 (log 2)")

if loss_ratio > 2.0:
    print("  → Discriminator is winning (generator struggling)")
elif loss_ratio < 0.5:
    print("  → Generator might be winning (discriminator struggling)")
else:
    print("  → Reasonably balanced training")

# Test final model performance
print(f"\n=== Final Model Performance ===")
with torch.no_grad():
    # Test on real data
    test_real = trainX[:100].to(device)
    real_scores = discriminator(test_real)
    
    # Test on fake data
    test_noise = torch.randn(100, noise_dim).to(device)
    fake_images = generator(test_noise)
    fake_scores = discriminator(fake_images)
    
    print(f"Discriminator performance:")
    print(f"  Real images score: {real_scores.mean().item():.3f} ± {real_scores.std().item():.3f}")
    print(f"  Fake images score: {fake_scores.mean().item():.3f} ± {fake_scores.std().item():.3f}")
    print(f"  Separation: {(real_scores.mean() - fake_scores.mean()).item():.3f}")
    
    if real_scores.mean() > 0.7 and fake_scores.mean() < 0.3:
        print("  → Discriminator still distinguishing well (may need more training)")
    elif 0.4 < real_scores.mean() < 0.6 and 0.4 < fake_scores.mean() < 0.6:
        print("  → Good equilibrium reached!")
    else:
        print("  → Training dynamics need adjustment")

## Noise-to-Image Mapping Analysis

Let's explore how GANs map random noise to structured images:

In [ ]:
print("=== Noise-to-Image Mapping Analysis ===")

# Demonstrate noise interpolation
def interpolate_noise(generator, z1, z2, steps=10):
    """Interpolate between two noise vectors"""
    alphas = torch.linspace(0, 1, steps)
    interpolations = []
    
    for alpha in alphas:
        z_interp = (1 - alpha) * z1 + alpha * z2
        with torch.no_grad():
            x_interp = generator(z_interp.unsqueeze(0))
            interpolations.append(x_interp.cpu().numpy())
    
    return interpolations

# Create noise interpolation
generator.eval()
with torch.no_grad():
    z1 = torch.randn(noise_dim).to(device)
    z2 = torch.randn(noise_dim).to(device)
    
    interpolations = interpolate_noise(generator, z1, z2, steps=10)
    
    # Visualize interpolation
    fig, axes = plt.subplots(2, 10, figsize=(15, 3))
    
    # First row: interpolation
    for i, interp in enumerate(interpolations):
        axes[0, i].imshow(interp.reshape(28, 28), cmap='gray')
        axes[0, i].set_xticks([])
        axes[0, i].set_yticks([])
        axes[0, i].set_title(f'α={i/9:.1f}')
    
    # Second row: random samples for comparison
    random_noise = torch.randn(10, noise_dim).to(device)
    random_images = generator(random_noise).cpu().numpy()
    
    for i in range(10):
        axes[1, i].imshow(random_images[i].reshape(28, 28), cmap='gray')
        axes[1, i].set_xticks([])
        axes[1, i].set_yticks([])
        if i == 0:
            axes[1, i].set_ylabel('Random')
    
    axes[0, 0].set_ylabel('Interpolation')
    plt.suptitle('GAN Noise Space Structure')
    plt.tight_layout()
    plt.show()

# Analyze noise sensitivity
print("\nNoise sensitivity analysis:")
base_noise = torch.randn(noise_dim).to(device)
base_image = generator(base_noise.unsqueeze(0))

# Add small perturbations
perturbations = [0.0, 0.1, 0.5, 1.0, 2.0]
perturbed_images = []

for eps in perturbations:
    perturbed_noise = base_noise + eps * torch.randn_like(base_noise)
    with torch.no_grad():
        perturbed_image = generator(perturbed_noise.unsqueeze(0))
        perturbed_images.append(perturbed_image.cpu().numpy())
        
        # Compute image difference
        diff = torch.mean((perturbed_image - base_image) ** 2).item()
        print(f"  Noise perturbation {eps:.1f} → Image MSE: {diff:.4f}")

# Visualize perturbations
fig, axes = plt.subplots(1, len(perturbations), figsize=(12, 2))
for i, (eps, img) in enumerate(zip(perturbations, perturbed_images)):
    axes[i].imshow(img.reshape(28, 28), cmap='gray')
    axes[i].set_xticks([])
    axes[i].set_yticks([])
    axes[i].set_title(f'ε = {eps:.1f}')

plt.suptitle('Effect of Noise Perturbations')
plt.tight_layout()
plt.show()

print("\n✓ GANs learn a smooth mapping from noise to images!")
print("✓ Small noise changes lead to gradual image changes")
print("✓ Large noise changes can produce completely different digits")

## Key Takeaways: GANs vs Other Generative Models

### Advantages of GANs
1. **Sharp, High-Quality Images**: No blurriness like VAEs
2. **No Explicit Likelihood**: Focuses purely on sample quality
3. **Flexible Architectures**: Can use any differentiable networks
4. **Direct Sampling**: Simple noise-to-image mapping
5. **Scalable**: Works well with large, complex datasets

### Limitations of GANs
1. **Training Instability**: Sensitive to hyperparameters, architecture
2. **Mode Collapse**: May generate limited variety
3. **No Likelihood**: Cannot evaluate probability of data
4. **Difficult Evaluation**: Requires external metrics (FID, IS)
5. **No Latent Inference**: Cannot encode data to latent space

### Training Challenges
1. **Nash Equilibrium**: Hard to reach stable equilibrium
2. **Gradient Issues**: Vanishing/exploding gradients
3. **Hyperparameter Sensitivity**: Learning rates, architectures
4. **Monitoring**: Loss values don't directly indicate quality

### Comparison Summary

| Aspect | GAN | VAE | Real NVP |
|--------|-----|-----|----------|
| **Sample Quality** | Excellent | Blurry | Good |
| **Training Stability** | Unstable | Stable | Stable |
| **Likelihood** | None | Lower bound | Exact |
| **Latent Space** | Not guaranteed | Meaningful | Meaningful |
| **Mode Coverage** | Can collapse | Good | Good |
| **Evaluation** | External metrics | Likelihood | Likelihood |

### Modern GAN Improvements
- **DCGAN**: Convolutional architectures
- **WGAN**: Wasserstein distance for stability
- **StyleGAN**: Style-based generation
- **Progressive GAN**: Progressive growing
- **Spectral Normalization**: Training stability

### Best Practices for GANs
- **Architecture**: Use DCGAN-style convolutions for images
- **Learning Rates**: Often different for G and D
- **Batch Normalization**: Helps training stability
- **Feature Matching**: Alternative generator objective
- **Multiple Updates**: Train D multiple times per G update

---

**References:**
- Goodfellow, I., et al. (2014). Generative adversarial nets. NIPS.
- Radford, A., Metz, L., & Chintala, S. (2015). Unsupervised representation learning with deep convolutional generative adversarial networks. arXiv preprint arXiv:1511.06434.
- Arjovsky, M., Chintala, S., & Bottou, L. (2017). Wasserstein generative adversarial networks. ICML.